In [ ]:
import importlib, data
importlib.reload(data)



In [ ]:
import pandas as pd
import numpy as np
import optuna
from darts import TimeSeries
from darts.models import TCNModel
from darts.metrics import mape
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import ScalerWrapper
from darts.dataprocessing.transformers import Scaler

def prepare_series(returns: pd.Series, volume: pd.Series):
    ts_returns = TimeSeries.from_series(returns)
    ts_volume = TimeSeries.from_series(volume)
    
    # Optional: scale both series
    scaler_returns = Scaler()
    scaler_volume = Scaler()

    ts_returns_scaled = scaler_returns.fit_transform(ts_returns)
    ts_volume_scaled = scaler_volume.fit_transform(ts_volume)

    return ts_returns_scaled, [ts_volume_scaled], scaler_returns


def objective(trial, returns: pd.Series, volume: pd.Series):
    ts_target, ts_covariates, scaler_target = prepare_series(returns, volume)

    # Split (80-20)
    train, val = ts_target[:-20], ts_target[-20:]
    cov_train, cov_val = ts_covariates[0][:-20], ts_covariates[0][-20:]

    model = TCNModel(
        input_chunk_length=trial.suggest_int("input_chunk_length", 12, 60),
        output_chunk_length=1,
        n_epochs=trial.suggest_int("n_epochs", 50, 200),
        kernel_size=trial.suggest_int("kernel_size", 2, 6),
        num_filters=trial.suggest_int("num_filters", 3, 32),
        dilation_base=trial.suggest_int("dilation_base", 2, 8),
        weight_norm=trial.suggest_categorical("weight_norm", [True, False]),
        dropout=trial.suggest_float("dropout", 0.0, 0.3),
        random_state=42,
        likelihood=None,
        verbose=False
    )

    model.fit(train, past_covariates=cov_train, verbose=False)
    preds = model.predict(n=20, past_covariates=cov_val)

    return mape(val, preds)


def rolling_predictions(returns: pd.Series, volume: pd.Series, model: TCNModel, input_chunk_length: int):
    ts_target, ts_covariates, scaler_target = prepare_series(returns, volume)
    predictions = []
    timestamps = []

    for i in range(input_chunk_length, len(ts_target) - 1):
        past_target = ts_target[i - input_chunk_length:i]
        past_cov = ts_covariates[0][i - input_chunk_length:i + 1]  # covs include current time
        model.fit(past_target, past_covariates=past_cov, verbose=False)
        pred = model.predict(n=1, past_covariates=past_cov)
        predictions.append(pred.values()[0][0])
        timestamps.append(ts_target.time_index[i])

    # Convert predictions back to original scale
    pred_series = TimeSeries.from_times_and_values(timestamps, predictions)
    pred_series = scaler_target.inverse_transform(pred_series)

    return pred_series.pd_series()
